In [1]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from ir_measures import *
import pandas as pd

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#pt.init()
ensure_pyterrier_is_loaded()
tira = Client()


PyTerrier 0.10.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
pt_dataset = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-01-20240426-training")
pt_dataset_jun = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-06-20240418-training")
pt_dataset_aug = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-08-20240418-training")

In [5]:
bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-01-20240423-training')
sce = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', 'ir-benchmarks/longeval-2023-01-20240423-training')
cb = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-01-20240423-training')
rz = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', 'ir-benchmarks/longeval-2023-01-20240423-training')
fusion_jan = pt.transformer.get_transformer(pt.io.read_results('galapagos-tortoise-wsum.lag1'))
fusion_jun = pt.transformer.get_transformer(pt.io.read_results('galapagos-tortoise-wsum.lag6'))
fusion_aug = pt.transformer.get_transformer(pt.io.read_results('galapagos-tortoise-wsum.lag8'))

In [6]:
hypo_jan = pt.Experiment(
    [
        bm25,
        sce,
        cb,
        rz,
        fusion_jan,
    ],
    pt_dataset.get_topics('text'),
    pt_dataset.get_qrels(),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
        # "3-max-avg",
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
)

hypo_jan

,name,nDCG(judged_only=True)@10,nDCG(judged_only=True),nDCG@10,nDCG
0,BM25,0.444411,0.472638,0.199380,0.317285
1,Sparse Cross-Encoder,0.448109,0.474784,0.220869,0.336719
2,ColBERT,0.450091,0.476047,0.216457,0.330096
3,RankZephyr,0.455142,0.482076,0.246549,0.352920
4,WSum,0.454667,0.480725,0.251512,0.355077


In [7]:
bm25_jun = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-06-20240418-training')
sce_jun = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', 'ir-benchmarks/longeval-2023-06-20240418-training')
cb_jun = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-06-20240418-training')
rz_jun = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', 'ir-benchmarks/longeval-2023-06-20240418-training')

### Hypothesis Test on June-Dataset.

In [8]:
hypo_jun = pt.Experiment(
    [
        bm25_jun,
        sce_jun,
        cb_jun,
        rz_jun,
        fusion_jun,
    ],
    pt_dataset_jun.get_topics('text'),
    pt_dataset_jun.get_qrels(),
    #pd.read_csv("01_qrels_jun.txt"),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
    baseline = 4
)

hypo_jun


,name,nDCG,nDCG@10,nDCG(judged_only=True),nDCG(judged_only=True)@10,nDCG +,nDCG -,nDCG p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value,nDCG(judged_only=True) +,nDCG(judged_only=True) -,nDCG(judged_only=True) p-value,nDCG(judged_only=True)@10 +,nDCG(judged_only=True)@10 -,nDCG(judged_only=True)@10 p-value
0,BM25,0.252253,0.165902,0.387891,0.375125,97.0,234.0,3.387580e-17,74.0,213.0,1.294152e-17,95.0,154.0,0.001333,96.0,153.0,0.001136
1,Sparse Cross-Encoder,0.276560,0.201901,0.393611,0.381502,111.0,220.0,1.356805e-04,100.0,193.0,1.761921e-05,101.0,145.0,0.038635,96.0,149.0,0.034443
2,ColBERT,0.263776,0.183421,0.388463,0.376752,95.0,236.0,1.201873e-08,84.0,205.0,9.834654e-11,106.0,149.0,0.002078,104.0,151.0,0.003803
3,RankZephyr,0.294686,0.227905,0.400839,0.390550,135.0,176.0,3.441785e-01,118.0,120.0,9.810752e-01,84.0,86.0,0.615196,84.0,83.0,0.297015
4,WSum,0.292732,0.227834,0.399975,0.388633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
bm25_aug = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-08-20240418-training')
sce_aug = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', 'ir-benchmarks/longeval-2023-08-20240418-training')
cb_aug = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-08-20240418-training')
rz_aug = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', 'ir-benchmarks/longeval-2023-08-20240418-training')

The download is derived from The LongEval Dataset under the "Qwant LongEval Attribution-NonCommercial-ShareAlike License". Hence, the download is also under this License. By using it, you agree to the terms of this license. Please find details at: https://lindat.mff.cuni.cz/repository/xmlui/page/Qwant_LongEval_BY-NC-SA_License


Download: 2.48MiB [00:00, 17.5MiB/s]


Download finished. Extract...
Extraction finished:  /home/codespace/.tira/extracted_runs/ir-benchmarks/longeval-2023-08-20240418-training/fschlatt


### Hypothesis Test on August-Dataset.

In [10]:
hypo_aug = pt.Experiment(
    [
        bm25_aug,
        sce_aug,
        cb_aug,
        rz_aug,
        fusion_aug,
    ],
    pt_dataset_aug.get_topics('text'),
    pt_dataset_aug.get_qrels(),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
    baseline = 4
)

hypo_aug

,name,nDCG,nDCG@10,nDCG(judged_only=True),nDCG(judged_only=True)@10,nDCG +,nDCG -,nDCG p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value,nDCG(judged_only=True) +,nDCG(judged_only=True) -,nDCG(judged_only=True) p-value,nDCG(judged_only=True)@10 +,nDCG(judged_only=True)@10 -,nDCG(judged_only=True)@10 p-value
0,BM25,0.191457,0.140883,0.300187,0.296796,347.0,771.0,2.643216e-34,283.0,656.0,4.964397e-35,307.0,409.0,0.000034,307.0,403.0,0.000013
1,Sparse Cross-Encoder,0.211935,0.168767,0.305075,0.302068,447.0,654.0,2.137901e-05,379.0,547.0,8.867774e-06,329.0,358.0,0.375969,325.0,359.0,0.297648
2,ColBERT,0.205953,0.161361,0.305006,0.301849,416.0,676.0,3.051411e-11,361.0,546.0,2.981866e-11,346.0,384.0,0.412428,342.0,388.0,0.290758
3,RankZephyr,0.219348,0.177775,0.306502,0.303919,418.0,576.0,5.159930e-01,343.0,376.0,1.510246e-01,219.0,219.0,0.740464,213.0,218.0,0.655888
4,WSum,0.220011,0.179789,0.306251,0.303571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df = pd.DataFrame({#'system name':["BM25", "Sparse Cross-Encoder", "ColBERT", "RankZephyr", "WSum"],
                   'nDCG_01-06': [0,0,0,0,0],
                   'nDCG_06-08': [0,0,0,0,0],
                   'nDCG@10_01-06': [0,0,0,0,0],
                   'nDCG@10_06-08': [0,0,0,0,0],})
df["nDCG_01-06"] = hypo_jan["nDCG"] - hypo_jun["nDCG"]
df["nDCG_06-08"] = hypo_jun["nDCG"] - hypo_aug["nDCG"]
df["nDCG@10_01-06"] = hypo_jan["nDCG@10"] - hypo_jun["nDCG@10"]
df["nDCG@10_06-08"] = hypo_jun["nDCG@10"] - hypo_aug["nDCG@10"]

df

,nDCG_01-06,nDCG_06-08,nDCG@10_01-06,nDCG@10_06-08
0,0.065032,0.060796,0.033478,0.025019
1,0.060159,0.064626,0.018968,0.033134
2,0.066320,0.057823,0.033036,0.022060
3,0.058234,0.075338,0.018644,0.050129
4,0.062345,0.072722,0.023678,0.048044


In [27]:
#x=df.loc[0]
#df.corrwith(x, axis=0)
df.T.corr()

,0,1,2,3,4
0,1.000000,0.888821,0.995730,0.652626,0.745275
1,0.888821,1.000000,0.852826,0.924650,0.967595
2,0.995730,0.852826,1.000000,0.591431,0.693346
3,0.652626,0.924650,0.591431,1.000000,0.990185
4,0.745275,0.967595,0.693346,0.990185,1.000000
